1. Extract all data from CSV files.
2. Transform data into inputs/targets.
3. Split inputs/targets into train/test sets.
4. Sample inputs/targets for the testing sets.
5. Load outputs inputs/targets to parquet files.

# Imports 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Config 

In [ ]:

DATA_PATH = '../loan_data/credit_risk_dataset.csv'

# Load the dataframe

In [ ]:
train_data = pd.read_csv(DATA_PATH, index_col='id')

In [ ]:
feature_df = train_data[::-1]
target_df = train_data['loan_status']

# Isolate categorical data from numerical 

In [ ]:

numeric_columns = train_data.select_dtypes(include=[np.number])
categorical_columns = train_data.select_dtypes(exclude=[np.number])

# Create the proprocess pipeline 